## Desafio 1


Dada uma base de dados de imagens de celebridades na pasta "5-celebrity-faces-dataset", colete os dados de cada celebridade identificado por meio do seu nome, por exemplo "ben_afflek" e use a pasta "train" para treinar um modelo de classificador de rostos e valide os resultados com outras amostras que estão na pasta "val".

<img src="imagens/resultado.png">

In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import glob

In [2]:
face_classifier = cv2.CascadeClassifier('classificadores/haarcascade_frontalface_default.xml')

# Extrator de faces
def face_extractor(img):
    try:
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)

        if faces is ():
            return None

        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h, x:x+w]
            
    except:
        return None

    return cropped_face

In [3]:
nomes = ["ben_afflek", "elton_john", "jerry_seinfeld", "madonna"]
persons = {0: "Ben", 1: "Elton", 2: "Jerry", 3: "Madonna"}

In [4]:
def normalizar_imagens(nome):
    list_files = []
    i = 1
    no_face = 0
    total = 0
    for file in glob.glob("imagens/5-celebrity-faces-dataset/data/train/"+nome+"/*"):
        total+=1
        image = cv2.imread(file);
        face = face_extractor(image)
        if face is not None:
            image_norm = cv2.resize(face, (200, 200))
            image_norm = cv2.cvtColor(image_norm, cv2.COLOR_BGR2GRAY)
            file_name_path = "imagens/5-celebrity-faces-dataset/data/train/"+nome+"/norm/"+str(i)+".png"
            cv2.imwrite(file_name_path, image_norm)
            list_files.append(file_name_path)
            i += 1
        else:
            no_face += 1
    print("Faces localizadas "+str(i) + "/" + str(total))
    return list_files

In [5]:
def treinamento():
    training_data, labels = [], []

    for idx, nome in enumerate(nomes):
        print("Processando "+nome)
        files = normalizar_imagens(nome)

        # Lendo as imagens e associando a um label
        for i, file in enumerate(files):
            images = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
            training_data.append(np.asarray(images, dtype=np.uint8))
            labels.append(idx)

    # Criando uma matriz da lista de labels
    labels = np.asarray(labels, dtype=np.int32)
    return training_data, labels

In [6]:
training_data, labels = treinamento()

Processando ben_afflek
Faces localizadas 12/15
Processando elton_john
Faces localizadas 11/18
Processando jerry_seinfeld
Faces localizadas 22/22
Processando madonna
Faces localizadas 13/20


In [7]:
# Treinamento do modelo
model = cv2.face.LBPHFaceRecognizer_create()
model.train(np.asarray(training_data), labels)
print("Modelo treinado com sucesso.")

Modelo treinado com sucesso.


In [8]:
i = 1
no_face = 0
for idx, nome in enumerate(nomes):
    print("Analisando "+ nome)
    total = 0
    success = 0
    for file in glob.glob("imagens/5-celebrity-faces-dataset/data/val/"+nome+"/*"):
            total += 1
            image = cv2.imread(file);
            face = face_extractor(image)
            if face is not None:
                image_norm = cv2.resize(face, (200, 200))
                image_norm = cv2.cvtColor(image_norm, cv2.COLOR_BGR2GRAY)
                results = model.predict(image_norm)
                info = persons[results[0]] + " " + str(int(results[1]))+" pontos"
                print(info)
                if results[0] == idx:
                    success += 1
            else:
                no_face += 1
    print("Faces localizadas "+str(success) + "/" + str(total)+ " "+str(float(success)*100/float(total)) + "%")

Analisando ben_afflek
Jerry 54 pontos
Ben 52 pontos
Ben 25 pontos
Ben 79 pontos
Jerry 69 pontos
Faces localizadas 3/5 60.0%
Analisando elton_john
Elton 46 pontos
Madonna 76 pontos
Jerry 65 pontos
Jerry 64 pontos
Faces localizadas 1/5 20.0%
Analisando jerry_seinfeld
Jerry 47 pontos
Jerry 69 pontos
Jerry 43 pontos
Jerry 63 pontos
Jerry 59 pontos
Faces localizadas 5/5 100.0%
Analisando madonna
Jerry 75 pontos
Madonna 79 pontos
Jerry 54 pontos
Madonna 53 pontos
Jerry 69 pontos
Faces localizadas 2/5 40.0%


Analisando mais de um rosto ao mesmo tempo

In [10]:
def face_extractor_multi(img):
    list_faces = []
    list_xy = []
    
    try:
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)

        if faces is ():
            return None

        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h, x:x+w]
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
            face_dict = {"face": cropped_face, "coord_xy": (x,y)}
            list_faces.append(face_dict)
            
    except:
        return None

    return list_faces

In [11]:
image = cv2.imread("imagens/1.jpg");
faces = face_extractor_multi(image)
if faces is not None:
    for face in faces:
        image_norm = cv2.resize(face["face"], (200, 200))
        image_norm = cv2.cvtColor(image_norm, cv2.COLOR_BGR2GRAY)
        results = model.predict(image_norm)
        info = persons[results[0]] + " " + str(int(results[1]))+" pontos"
        cv2.putText(image, info, (face["coord_xy"][0], face["coord_xy"][1]-2), cv2.FONT_HERSHEY_DUPLEX, 1, (255,120,150), 1)
        cv2.imshow("Imagem Final com Digitos", image)
        cv2.waitKey(0) 
        cv2.destroyAllWindows()
        print(info)
else:
    print("Não encontrado faces")

Jerry 91 pontos
Ben 81 pontos
Madonna 75 pontos
